<a href="https://colab.research.google.com/github/infiniteoverflow/Generating-TV-Scripts/blob/master/Generate_TV_Scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 


In [0]:
# load in data
import helper
data_dir = 'Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data


In [2]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [0]:
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    counts = Counter(text)
    vocab = sorted(counts , key=counts.get , reverse=True)
    vocab_to_int = {word  : index for index,word in enumerate(vocab,1)}
    int_to_vocab = {index : word for index,word in enumerate(vocab,1)}
    # return tuple
    return (vocab_to_int, int_to_vocab)


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [0]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    punctuation = {}
    punctuation['.'] = "||Period||"
    punctuation[','] = "||Comma||"
    punctuation['"'] = "||Quotation_Mark||"
    punctuation[';'] = "||Semicolon||"
    punctuation['!'] = "||Exclamation_Mark||"
    punctuation['?'] = "||Question_Mark||"
    punctuation['('] = "||Left_Parentheses||"
    punctuation[')'] = "||Right_Parentheses||"
    punctuation['-'] = "||Dash||"
    punctuation['\n'] = "||Return||"
    return punctuation


## Preprocess all the data and save it

In [0]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [0]:
import helper

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Building the neural network

### Check GPU Access

In [0]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [0]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    features = np.zeros((len(words)-sequence_length+1,sequence_length) , dtype = int)
    targets = np.zeros((len(words)-sequence_length+1) ,dtype=int)
    for i in range(len(words)-sequence_length+1):
        features[i] = words[i:i+sequence_length]
        if i+sequence_length+1 < len(words) : 
            targets[i] = words[i+sequence_length] 
        else: 
            targets[i] = 0
        #print(targets[i])
    dataset = TensorDataset(torch.from_numpy(features) , torch.from_numpy(targets))
    dataloader = torch.utils.data.DataLoader(dataset , batch_size = batch_size)
    # return a dataloader
    return dataloader



---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

```

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout
        # define model layers
        self.embedding = nn.Embedding(vocab_size  , embedding_dim)
        self.recurrent = nn.LSTM(embedding_dim , hidden_dim,n_layers , dropout = dropout , batch_first=True)
        self.fc = nn.Linear(hidden_dim,output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        
        embedding_out = self.embedding(nn_input)
        LSTM_out , hidden = self.recurrent(embedding_out,hidden)
        
        # stack the output of Lstm
        LSTM_out = LSTM_out.contiguous().view(-1,self.hidden_dim)
        
        out = self.fc(LSTM_out)
        out = out.view(batch_size, -1, self.output_size)
        out = out[:,-1]
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        # initialize hidden state with zero weights, and move to GPU if available
        
        return hidden

In [0]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    hidden = tuple([each.data for each in hidden])
    
    # move data to GPU, if available
    if train_on_gpu :
        inp , target  = inp.cuda() , target.cuda() 
        
    # perform backpropagation and optimization
    optimizer.zero_grad()
    output , hidden = rnn.forward(inp,hidden)
    
    loss = criterion(output,target)
    loss.backward()
    
    nn.utils.clip_grad_norm_(rnn.parameters(), 3)
    optimizer.step()
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden


## Training

In [0]:

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

## Hyperparameters

In [0]:
# Data params
# Sequence Length
sequence_length = 8  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)


In [0]:
# Training parameters
# Number of Epochs
num_epochs = 10  # first 10
# Learning Rate
learning_rate = 0.0005

# Model parameters
# Vocab size
vocab_size = len({word for word in text.split()})
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300 # first 350
# Hidden Dimension
hidden_dim = 512  # first 450
# Number of RNN Layers
n_layers = 2  # first 3 

# Show stats for every n number of batches
show_every_n_batches = 500

In [14]:

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.607097013950348

Epoch:    1/10    Loss: 4.965666211605072

Epoch:    1/10    Loss: 4.725623914718628

Epoch:    1/10    Loss: 4.5929696111679075

Epoch:    1/10    Loss: 4.590596037387848

Epoch:    1/10    Loss: 4.608037612915039

Epoch:    1/10    Loss: 4.507072505950927

Epoch:    1/10    Loss: 4.383568517684936

Epoch:    1/10    Loss: 4.34893147277832

Epoch:    1/10    Loss: 4.290713621616364

Epoch:    1/10    Loss: 4.3992066555023195

Epoch:    1/10    Loss: 4.432708764076233

Epoch:    1/10    Loss: 4.420651005744934

Epoch:    2/10    Loss: 4.19120622690002

Epoch:    2/10    Loss: 4.000533264160156

Epoch:    2/10    Loss: 3.9014825644493105

Epoch:    2/10    Loss: 3.8626864595413206

Epoch:    2/10    Loss: 3.9092188396453857

Epoch:    2/10    Loss: 3.979631565093994

Epoch:    2/10    Loss: 3.9120251212120056

Epoch:    2/10    Loss: 3.8167380657196044

Epoch:    2/10    Loss: 3.807844075202942

Epoch:    2/10    Los

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Generate TV Script

In [0]:

import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq.cpu(), -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [21]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: you are the moops?

hoyt: so what do you think?

jerry: it's a misprint that you can contemplate it.

hoyt: call yev kassem.

george: what?

hoyt: you know, the only thing that could get us chiles.

[new witness: police detective]

hoyt: the judge is the most score.

hoyt: call a lot of junk to the defendants?

elaine: i think i can get you amoral- alecky?

hoyt: so, you know, they have the same trial. the whole thing is, i was a little.

hoyt: i don't have the parade, i don't have to go back to the bathroom.

george: i can't find it.

vandelay: the contest?

hoyt: i don't know, i don't want to know what this means is, you know, the only ones it is.

jerry: you know, i think i could see the testimony in my life that was the badly!

jerry: what is that?

george: yes, i think i know that the rumor.

jerry: i can't get the money.

jerry: so you can get it, jerry.

hoyt: i don't know.

elaine: i don't know.

jerry: i know that he was in latham and then the judge were flipped.

hoyt:

In [0]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()